# Value Iteration

## Import library & Create modified environment

In [1]:
import gym
from gym.envs.registration import register
# DO NOT RUN TWICE OR YOU'LL GET AN ERROR => SOLVE BY RESTARTING RUNTIME ("Runtime" -> "Restart Runtime")
register(id='FrozenLakeDeterministic-v0', 
         entry_point='gym.envs.toy_text:FrozenLakeEnv', 
         kwargs={'map_name': '4x4', 
                 'is_slippery': False}, 
         max_episode_steps=100)

## Modified FrozenLake-v0's MDP

In [2]:
env = gym.make('FrozenLakeDeterministic-v0')
print("Observation space:\t{}".format(env.observation_space))
print("Action space:\t\t{}".format(env.action_space))

print('-'*20)

num_states = env.nS
print("Number of states: {}".format(num_states))
num_actions = env.nA
print("Number of actions: {}".format(num_actions))

print('-'*20)

all_states = [i for i in range(num_states)]
print("All states: {}".format(all_states))
all_actions = [i for i in range(num_actions)]
print("All actions: {}".format(all_actions))

print('-'*20)

probability_matrix = env.P
print("Transition matrix:")
print(env.P)
print("Transition matrix with current state = 0")
print(env.P[0])
print("Transition matrix with current state = 0, action = 0")
print(env.P[0][0])
print("Note that this environment is deterministic! That is, for a given state and action, the next state is determined!")

print('-'*20)
print("4x4 map")
obs = env.render()

Observation space:	Discrete(16)
Action space:		Discrete(4)
--------------------
Number of states: 16
Number of actions: 4
--------------------
All states: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
All actions: [0, 1, 2, 3]
--------------------
Transition matrix:
{0: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 4, 0.0, False)], 2: [(1.0, 1, 0.0, False)], 3: [(1.0, 0, 0.0, False)]}, 1: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 5, 0.0, True)], 2: [(1.0, 2, 0.0, False)], 3: [(1.0, 1, 0.0, False)]}, 2: {0: [(1.0, 1, 0.0, False)], 1: [(1.0, 6, 0.0, False)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 2, 0.0, False)]}, 3: {0: [(1.0, 2, 0.0, False)], 1: [(1.0, 7, 0.0, True)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 3, 0.0, False)]}, 4: {0: [(1.0, 4, 0.0, False)], 1: [(1.0, 8, 0.0, False)], 2: [(1.0, 5, 0.0, True)], 3: [(1.0, 0, 0.0, False)]}, 5: {0: [(1.0, 5, 0, True)], 1: [(1.0, 5, 0, True)], 2: [(1.0, 5, 0, True)], 3: [(1.0, 5, 0, True)]}, 6: {0: [(1.0, 5, 0.0, True)], 1: [(1.0, 10, 0.0, False)], 2:

## Random Action

In [4]:
env = gym.make('FrozenLakeDeterministic-v0')
obs = env.reset()
total_reward = 0
num_steps = 0
while True:
    print('#'*10)
    print("Timestep {}".format(num_steps))
    print("Observation: {}".format(obs))
    print('#'*10)
    env.render()
    random_action = env.action_space.sample()
    print('#'*10)
    print("Action: {}".format(random_action))
    print('#'*10)
    obs, reward, done, info = env.step(random_action)
    total_reward += reward
    num_steps += 1
    print('-'*15)
    if done is True:
        print("Done!")
        break
env.render()
env.close()
print("Total reward of {:.2f} for {} number of steps".format(total_reward, num_steps))

##########
Timestep 0
Observation: 0
##########

SFFF
FHFH
FFFH
HFFG
##########
Action: 1
##########
---------------
##########
Timestep 1
Observation: 4
##########
  (Down)
SFFF
FHFH
FFFH
HFFG
##########
Action: 1
##########
---------------
##########
Timestep 2
Observation: 8
##########
  (Down)
SFFF
FHFH
FFFH
HFFG
##########
Action: 2
##########
---------------
##########
Timestep 3
Observation: 9
##########
  (Right)
SFFF
FHFH
FFFH
HFFG
##########
Action: 0
##########
---------------
##########
Timestep 4
Observation: 8
##########
  (Left)
SFFF
FHFH
FFFH
HFFG
##########
Action: 0
##########
---------------
##########
Timestep 5
Observation: 8
##########
  (Left)
SFFF
FHFH
FFFH
HFFG
##########
Action: 3
##########
---------------
##########
Timestep 6
Observation: 4
##########
  (Up)
SFFF
FHFH
FFFH
HFFG
##########
Action: 2
##########
---------------
Done!
  (Right)
SFFF
FHFH
FFFH
HFFG
Total reward of 0.00 for 7 number of steps


## Value Iteration Algorithm
For all states, <br>
$V_{0}(s)=0$<br>
$V_{t+1}(s)=R(s)+\gamma\max_{a\in A(s)}\sum_{s'}P(s'|s, a)V_{t}(s')$<br>
NOTE: In this modified environment, $P(s'|s,a)=1$ for all states and actions $s,a$<br>
Thus, for this special case, we can simply use the following update:<br>
$V_{t+1}(s)=R(s)+\gamma\max_{a\in A(s)}V_{t}(s'(s,a))$<br>
, where $s'$ is simply the next state returned given some state and action pair

In [5]:
# Initialize values and optimal policy for all states
values = {}
optimal_policy = {}
for state in all_states:
    values[state] = 0
    optimal_policy[state] = None

print("Current value table:")
print('\t', values)
print("Current optimal policy table:")
print('\t', optimal_policy)

Current value table:
	 {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}
Current optimal policy table:
	 {0: None, 1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None}


In [7]:
# Value iteration
env = gym.make('FrozenLakeDeterministic-v0')
env.reset()

num_iters = 1000
gamma = 0.99

from copy import deepcopy
import time
import math

start_time = time.time()
for it in range(num_iters):
    old_values = deepcopy(values)
    for state in all_states:
        best_value = -math.inf
        best_action = None
        for action in all_actions:
            ##### TO IMPLEMENT #####
            prob, next_state, reward, done = env.P[state][action][0]
            new_value = reward + gamma * old_values[next_state]
            if new_value > best_value:
                best_value = new_value
                best_action = action
                values[state] = best_value
                optimal_policy[state] = best_action
            ########################
    if (it+1) % 100 == 0:
        time_taken = time.time() - start_time
        print("Iteration: {}/{} \t\t{:.4f}s".format(it+1, num_iters, time_taken))

print('-'*40)
print("Current value table:")
print('\t', values)
print("Current optimal policy table:")
print('\t', optimal_policy)

Iteration: 100/1000 		0.0067s
Iteration: 200/1000 		0.0135s
Iteration: 300/1000 		0.0202s
Iteration: 400/1000 		0.0269s
Iteration: 500/1000 		0.0337s
Iteration: 600/1000 		0.0405s
Iteration: 700/1000 		0.0475s
Iteration: 800/1000 		0.0545s
Iteration: 900/1000 		0.0621s
Iteration: 1000/1000 		0.0705s
----------------------------------------
Current value table:
	 {0: 0.9516311429240483, 1: 0.9612371030240482, 2: 0.9709400930240484, 3: 0.9612371030240482, 4: 0.9612371030240482, 5: 0.00021369767468275617, 6: 0.9807410930240482, 7: 0.00029917674455585903, 8: 0.9709400930240484, 9: 0.9807410930240482, 10: 0.9906410930240481, 11: 0.0004701348843020647, 12: 0.0005128744192386156, 13: 0.9906410930240481, 14: 1.0006410930240484, 15: 0.0006410930240482708}
Current optimal policy table:
	 {0: 1, 1: 2, 2: 1, 3: 0, 4: 1, 5: 0, 6: 1, 7: 0, 8: 2, 9: 1, 10: 1, 11: 0, 12: 0, 13: 2, 14: 2, 15: 0}


In [8]:
# Evaluation
env = gym.make('FrozenLakeDeterministic-v0')
state = env.reset()
total_reward = 0
num_steps = 0
while True:
    print('#'*10)
    print("Timestep {}".format(num_steps))
    print("Observation: {}".format(obs))
    print('#'*10)
    env.render()
    optimal_action = optimal_policy[state]
    print('#'*10)
    print("Optimal action: {}".format(optimal_action))
    print('#'*10)
    state, reward, done, info = env.step(optimal_action)
    total_reward += reward
    num_steps += 1
    print('-'*15)
    if done is True:
        print("Done!")
        break
env.close()
print("Total reward of {:.2f} for {} number of steps".format(total_reward, num_steps))

##########
Timestep 0
Observation: 5
##########

SFFF
FHFH
FFFH
HFFG
##########
Optimal action: 1
##########
---------------
##########
Timestep 1
Observation: 5
##########
  (Down)
SFFF
FHFH
FFFH
HFFG
##########
Optimal action: 1
##########
---------------
##########
Timestep 2
Observation: 5
##########
  (Down)
SFFF
FHFH
FFFH
HFFG
##########
Optimal action: 2
##########
---------------
##########
Timestep 3
Observation: 5
##########
  (Right)
SFFF
FHFH
FFFH
HFFG
##########
Optimal action: 1
##########
---------------
##########
Timestep 4
Observation: 5
##########
  (Down)
SFFF
FHFH
FFFH
HFFG
##########
Optimal action: 2
##########
---------------
##########
Timestep 5
Observation: 5
##########
  (Right)
SFFF
FHFH
FFFH
HFFG
##########
Optimal action: 2
##########
---------------
Done!
Total reward of 1.00 for 6 number of steps


## (Optional) Solve bigger map!

In [ ]:
# Try with the following map!
random_map = gym.envs.toy_text.frozen_lake.generate_random_map(size=64)
env = gym.make("FrozenLakeDeterministic-v0", desc=random_map)
env.reset()
env.render()